# Algorithms in Comp Bio & BioInformatics
## 3rd_Assignment--GOR III Secondary Structure Prediction
# Moamin Abdulkareem

In [42]:
import numpy as np
import math 


In [3]:
def load_data(filename):
    
    f = open(filename)
    
    L = list()
    
    for line in f:
        
        line = line.split()
        
        if not line:
            continue
        
        L.append(line)
    
    L = np.asarray(L)
    return L

In [6]:
dssp_file = 'dssp_info.txt'
stride_file = 'stride_info.txt'
cath_file = 'cath_info.txt'
stride_dataset = load_data(stride_file)
dssp_dataset = load_data(dssp_file)
cath_dataset = load_data(cath_file)

print(len(stride_dataset))
print(len(dssp_dataset))
print(dssp_dataset)

71077
71391
[['1w0n' 'A' '12' 'ILE' 'Coil']
 ['1w0n' 'A' '13' 'THR' 'Beta']
 ['1w0n' 'A' '14' 'LYS' 'Beta']
 ...
 ['1ow4' 'A' '116' 'ASN' 'Helix']
 ['1ow4' 'A' '117' 'SER' 'Coil']
 ['1ow4' 'A' '118' 'TYR' 'Coil']]


In [8]:
# pre-process the datasets
amino_acids = ['ALA', 'ARG', 'ASN', 'ASP', 'CYS', 'GLU', 'GLN', 'GLY','HIS','ILE', 'LEU', 'LYS', 'MET','PHE', 'PRO','SER', 'THR', 'TRP', 'TYR', 'VAL']

def change_other_to_coil(dataset, amino_set):
    for i in range(len(dataset)):
        
        if dataset[i][4] == 'Other':
            dataset[i][4] = 'Coil'
    
    return dataset
stride = change_other_to_coil(stride_dataset, amino_acids)
dssp = change_other_to_coil(dssp_dataset, amino_acids)



amino_acids_letters = 'A,R,N,D,C,E,Q,G,H,I,L,K,M,F,P,S,T,W,Y,V'
amino_acids_letters = amino_acids_letters.split(',')

indices = list()
def remove_non_amino(dataset,amino_list):
    
    for i in range(len(dataset)):
        if dataset[i][3] not in amino_acids:
            indices.append(i)
    dataset = np.delete(dataset, indices, axis=0)
    return dataset
stride = remove_non_amino(stride, amino_acids)
dssp = remove_non_amino(dssp, amino_acids)


## Implementation of GOR III algorithm
### Using the formula below we have to compute the frequencies and make a sparse matrix of it
<img src="http://latex.codecogs.com/gif.latex?\dpi{120}&space;I(\Delta&space;S_j;&space;R_1,...,&space;R_n)&space;=&space;log(f_{S_j,&space;R_j&plus;m,&space;R_j}&space;/&space;f_{n-S_j,&space;R_j&plus;m,&space;R_j})&space;&plus;&space;log(f_{n-S_j,R_j}&space;/&space;f_{S_j,R_j})" title="I(\Delta S_j; R_1,..., R_n) = log(f_{S_j, R_j+m, R_j} / f_{n-S_j, R_j+m, R_j}) + log(f_{n-S_j,R_j} / f_{S_j,R_j})" />


In [ ]:
# since the dataset is huge so i made this function to extract a specific number of proteins
# then we can apply the algorithm on this portion of the dataset
def get_n_proteins(dataset, n):
    
    protein_count = 0
    start = dataset[0][0]
    
    alist = list()
    proteins_names = list()
    j = 0
    i = 0
    while i < len(dataset):
        
        protein_count += 1
        proteins_names.append(start)
        while dataset[j][0] == start and protein_count < n and j < len(dataset):
            alist.append(dataset[j])
            
            j += 1
        start = dataset[j][0]
        i += j
    return alist, proteins_names


In [73]:
# Here I start with the implementation of the algorithm based on the equation above
# We have to compute the self information and then the directional information using these formulas above

dssp_20, first_20_proteins = get_n_proteins(dssp, 20)
# Now i will use this function to get the frequencies per protein so 
def get_frequencies_of_conformations(dataset):
    
    Helix_freq = 0
    Beta_freq = 0
    Coil_freq = 0
    
    for i in range(len(dataset)):
            
        if dataset[i][4] == 'Helix':
            
            Helix_freq +=1
            
        elif dataset[i][4] == 'Beta':
            
            Beta_freq += 1
            
        else:
            Coil_freq += 1
        
        # The output of this function will be the frequencies of every conformation
        # And also the frequencies fn-s which is observing non specific conformation
        
    total_freq = Helix_freq + Beta_freq + Coil_freq
    non_helix_freq = total_freq - Helix_freq
    non_Beta_freq = total_freq - Beta_freq
    non_coil_freq = total_freq - Coil_freq
        
    return Helix_freq, non_helix_freq, Beta_freq, non_Beta_freq, Coil_freq, non_coil_freq




In [74]:
def get_f_s_r(dataset, protein_name):
    
    freq_dict = {}
        
    for amino_acid in amino_acids:
        count_helix = 0
        count_sheet = 0
        count_coil = 0
        count_total = 0
        for line in dataset:
            if line[0] == protein_name:
                if line[3] == amino_acid and line[4] == 'Helix':
                    count_helix +=1
                elif line[3] == amino_acid and line[4] == 'Beta':
                    count_sheet +=1
                elif line[3] == amino_acid and line[4] == 'Coil':
                    count_coil +=1
                count_total = count_helix + count_sheet + count_coil
        
        # we counted the total so we can get fn-s_r also in the output
        # the output will be a tuple which consists of two values
        # the first thing is the count of a specific conformation for a specific amino_acid
        # the second element in the tuple is the number of times we don't encounter a specific
        # conformation for a specific aminoacid
        freq_dict[(amino_acid, 'Helix')] = (count_helix, count_total - count_helix)
        freq_dict[(amino_acid, 'Beta')] = (count_sheet, count_total - count_sheet)
        freq_dict[(amino_acid, 'Coil')] = (count_coil, count_total - count_coil)
    
    # to count number of times we see a specific conformation and also the number of times that we don't observe it
        
    return freq_dict 
f_s_r = get_f_s_r(dssp,'1w0n')

print(f_s_r[('ALA', 'Helix')])


(0, 13)


In [79]:
# Now we will compute the self information and then the directional information

dssp_20, first_20_proteins = get_n_proteins(dssp,20)
stride_20, first_20_stride = get_n_proteins(stride, 20)

def get_self_information(dataset, protein_list):
    
    # we have to get the frequencies of the conformations in the dataset first
    self_information = list()
    
    i = 0
    
    f_helix, f_non_helix, f_beta, f_non_beta, f_coil, f_non_coil = get_frequencies_of_conformations(dataset)
    
    dataset = get_n_proteins(dataset,len(protein_list))
    
    while i < len(protein_list):
        
        print(f_helix, f_beta, f_coil)
        f_s_r = get_f_s_r(dataset, protein_list[i])
        
        for line in dataset:
            
            amino_acid = line[3]
            
            if line[0] == protein_list[i]:
                
        
                I_helix = m.log(f_s_r[(amino_acid, 'Helix')][0] / f_s_r[(amino_acid, 'Helix')][1] ) + m.log(f_non_helix/f_helix)
                I_beta = m.log(f_s_r[(amino_acid, 'Beta')][0] / f_s_r[(amino_acid, 'Beta')][1]) + m.log(f_non_beta/ f_beta)
                I_coil = m.log(f_s_r[(amino_acid, 'Coil')][0] / f_s_r[(amino_acid, 'Coil')][1]) + m.log(f_non_coil / f_coil)
                
                self_information.append([I_helix,I_beta, I_coil])
            else:
                i += 1
                
                break
        
    return self_information
self_information_dssp = get_self_information(dssp, first_20_stride)

print(self_information_dssp)

AttributeError: 'list' object has no attribute 'tolist'

In [71]:
# Implementation on DSSP file - datafromfile1 list
datafromfile1 = dssp
numb_check = ""
protein_num = 0
final_dssp = []

for l in datafromfile1: # looping over the lines 
    if l[0] != numb_check and protein_num<20 : # protein check 
        numb_check = l[0]
        protein_id = l[0]
        count = 0
        protein_num = protein_num + 1
        for line in datafromfile1: # looping over the lines second time a
            if line[0] == protein_id: # checking the protein 
                if line[3] != 'null':
                    aminoacid = line[3] # now we take the amino acid
                    count_total_aa = 0
                    count_helix = 0
                    count_sheet = 0
                    count_coil = 0
                    this_count_helix = 0
                    this_count_sheet = 0
                    this_count_coil = 0
                    for line in datafromfile1: # looping for the third time to calculate the conformations
                        
                        # first we check the whole conformaions in a protein
                        # because we are checking the protein id again
                        
                        if line[0]!= protein_id: #leave one out for validation
                            count_total_aa = count_total_aa + 1
                            if line[4]=='Helix':
                                count_helix = count_helix + 1
                            if line[4]=='Beta':
                                count_sheet = count_sheet + 1
                            if line[4]=='Coil':
                                count_coil = count_coil + 1
                            # inside the protein we match with the amino acid and we count the conformation for aminoacid
                            
                            if line[4] == 'Helix' and line[3] == aminoacid:
                                this_count_helix = this_count_helix + 1
                            elif line[4] == 'Beta' and line[3] == aminoacid:
                                this_count_sheet = this_count_sheet + 1
                            elif line[4] == 'Coil' and line[3] == aminoacid:
                                this_count_coil = this_count_coil + 1
                    # outside the third loop we get the final result
                    # note that the third loop doesn't work if the protein id changes so it gets out of the loop
                    
                    fsr1 = this_count_helix
                    fsr2 = this_count_sheet
                    fsr3 = this_count_coil

                    fsr_total = fsr1 + fsr2 + fsr3

                    fn_sr1 = fsr_total - this_count_helix
                    fn_sr2 = fsr_total - this_count_sheet
                    fn_sr3 = fsr_total - this_count_coil

                    fs1 = count_helix
                    fs2 = count_sheet
                    fs3 = count_coil
                    print(fs1)
                    print(fs2)
                    print(fs3)
                    fn_s1 = count_total_aa - count_helix
                    fn_s2 = count_total_aa - count_sheet
                    fn_s3 = count_total_aa - count_coil

                    '''
                    print (count_total_aa)
                    print (count_helix)
                    print(count_sheet)
                    print(count_coil)
                    print(this_count_helix)
                    print(this_count_sheet)
                    print(this_count_coil)
                    print("------")'''
                    # this is performed on the amino acid and then the count of conformation based on the protein
                    Ih = math.log(fsr1/fn_sr1) + math.log(fn_s1/fs1)
                    Is = math.log(fsr2/fn_sr2) + math.log(fn_s2/fs2)
                    Ic = math.log(fsr3/fn_sr3) + math.log(fn_s3/fs3)

                    # Printing self information
                    #print(Ih)
                    #print(Is)
                    #print(Ic)
                    #calculation of self information
                    print ("Self Information %3f, %f, %f  " % (Ih , Is, Ic))



                    count = count +1
                    print(count)

                    #calculating total residue in this protein
                    calculation = 0 
                    arrayforprotein = []
                    counterforarrayforprotein = 1
                    for calc in datafromfile1:
                        if calc[0] == protein_id and calc[3] != 'null':
                            calculation = calculation + 1
                            arrayforprotein.append(calc)

                    Ih1 = 0
                    Is1 = 0 
                    Ic1 = 0
                    
                    #calculation pair information if left and right side we have more than 8 residues
                    if count > 8 and calculation - count > 8 :

                        for i in range (count-8,count+9):
                            if i!= count:
                                if arrayforprotein[i][3] != 'null':
                                        aminoacid = arrayforprotein[i][3]
                                        count_total_aa = 0
                                        count_helix = 0
                                        count_sheet = 0
                                        count_coil = 0
                                        this_count_helix = 0
                                        this_count_sheet = 0
                                        this_count_coil = 0
                                        for line in datafromfile1:
                                            if line[0]!= protein_id:
                                                count_total_aa = count_total_aa + 1
                                                if line[4]=='Helix':
                                                    count_helix = count_helix + 1
                                                if line[4]=='Beta':
                                                    count_sheet = count_sheet + 1
                                                if line[4]=='Coil':
                                                    count_coil = count_coil + 1

                                                if line[4] == 'Helix' and line[3] == aminoacid:
                                                    this_count_helix = this_count_helix + 1
                                                elif line[4] == 'Beta' and line[3] == aminoacid:
                                                    this_count_sheet = this_count_sheet + 1
                                                elif line[4] == 'Coil' and line[3] == aminoacid:
                                                    this_count_coil = this_count_coil + 1

                                        fsr1 = this_count_helix
                                        fsr2 = this_count_sheet
                                        fsr3 = this_count_coil

                                        fsr_total = fsr1 + fsr2 + fsr3

                                        fn_sr1 = fsr_total - this_count_helix
                                        fn_sr2 = fsr_total - this_count_sheet
                                        fn_sr3 = fsr_total - this_count_coil

                                        fs1 = count_helix
                                        fs2 = count_sheet
                                        fs3 = count_coil

                                        fn_s1 = count_total_aa - count_helix
                                        fn_s2 = count_total_aa - count_sheet
                                        fn_s3 = count_total_aa - count_coil

                                        '''
                                        print (count_total_aa)
                                        print (count_helix)
                                        print(count_sheet)
                                        print(count_coil)
                                        print(this_count_helix)
                                        print(this_count_sheet)
                                        print(this_count_coil)
                                        print("------")'''

                                        Ih1 = math.log(fsr1/fn_sr1) + math.log(fn_s1/fs1) + Ih1
                                        Is1 = math.log(fsr2/fn_sr2) + math.log(fn_s2/fs2) + Is1
                                        Ic1 = math.log(fsr3/fn_sr3) + math.log(fn_s3/fs3) + Ic1

                                        # Printing self information
                                        #print(Ih)
                                        #print(Is)
                                        #print(Ic)
                                        #print("--------")


                    # calculating pair information if left side we have more than 8 and right side less than 8
                    elif count > 8:
                        for i in range(count-8,calculation):
                            if i!= count:
                                if arrayforprotein[i][3] != 'null':
                                        aminoacid = arrayforprotein[i][3]
                                        count_total_aa = 0
                                        count_helix = 0
                                        count_sheet = 0
                                        count_coil = 0
                                        this_count_helix = 0
                                        this_count_sheet = 0
                                        this_count_coil = 0
                                        for line in datafromfile1:
                                            if line[0]!= protein_id:
                                                count_total_aa = count_total_aa + 1
                                                if line[4]=='Helix':
                                                    count_helix = count_helix + 1
                                                if line[4]=='Beta':
                                                    count_sheet = count_sheet + 1
                                                if line[4]=='Coil':
                                                    count_coil = count_coil + 1

                                                if line[4] == 'Helix' and line[3] == aminoacid:
                                                    this_count_helix = this_count_helix + 1
                                                elif line[4] == 'Beta' and line[3] == aminoacid:
                                                    this_count_sheet = this_count_sheet + 1
                                                elif line[4] == 'Coil' and line[3] == aminoacid:
                                                    this_count_coil = this_count_coil + 1

                                        fsr1 = this_count_helix
                                        fsr2 = this_count_sheet
                                        fsr3 = this_count_coil

                                        fsr_total = fsr1 + fsr2 + fsr3

                                        fn_sr1 = fsr_total - this_count_helix
                                        fn_sr2 = fsr_total - this_count_sheet
                                        fn_sr3 = fsr_total - this_count_coil

                                        fs1 = count_helix
                                        fs2 = count_sheet
                                        fs3 = count_coil

                                        fn_s1 = count_total_aa - count_helix
                                        fn_s2 = count_total_aa - count_sheet
                                        fn_s3 = count_total_aa - count_coil

                                        '''
                                        print (count_total_aa)
                                        print (count_helix)
                                        print(count_sheet)
                                        print(count_coil)
                                        print(this_count_helix)
                                        print(this_count_sheet)
                                        print(this_count_coil)
                                        print("------")'''

                                        Ih1 = math.log(fsr1/fn_sr1) + math.log(fn_s1/fs1) + Ih1
                                        Is1 = math.log(fsr2/fn_sr2) + math.log(fn_s2/fs2) + Is1
                                        Ic1 = math.log(fsr3/fn_sr3) + math.log(fn_s3/fs3) + Ic1

                                        # Printing self information
                                        #print(Ih)
                                        #print(Is)
                                        #print(Ic)
                                        #print("--------")
                    #calculation pair information when left and right both sides we have less than 8 residues
                    elif count < 8 and calculation - count > 8:
                        for i in range(1,count+9):
                            if i!= count:
                                if arrayforprotein[i][3] != 'null':
                                        aminoacid = arrayforprotein[i][3]
                                        count_total_aa = 0
                                        count_helix = 0
                                        count_sheet = 0
                                        count_coil = 0
                                        this_count_helix = 0
                                        this_count_sheet = 0
                                        this_count_coil = 0
                                        for line in datafromfile1:
                                            if line[0]!= protein_id:
                                                count_total_aa = count_total_aa + 1
                                                if line[4]=='Helix':
                                                    count_helix = count_helix + 1
                                                if line[4]=='Beta':
                                                    count_sheet = count_sheet + 1
                                                if line[4]=='Coil':
                                                    count_coil = count_coil + 1

                                                if line[4] == 'Helix' and line[3] == aminoacid:
                                                    this_count_helix = this_count_helix + 1
                                                elif line[4] == 'Beta' and line[3] == aminoacid:
                                                    this_count_sheet = this_count_sheet + 1
                                                elif line[4] == 'Coil' and line[3] == aminoacid:
                                                    this_count_coil = this_count_coil + 1

                                        fsr1 = this_count_helix
                                        fsr2 = this_count_sheet
                                        fsr3 = this_count_coil

                                        fsr_total = fsr1 + fsr2 + fsr3

                                        fn_sr1 = fsr_total - this_count_helix
                                        fn_sr2 = fsr_total - this_count_sheet
                                        fn_sr3 = fsr_total - this_count_coil

                                        fs1 = count_helix
                                        fs2 = count_sheet
                                        fs3 = count_coil

                                        fn_s1 = count_total_aa - count_helix
                                        fn_s2 = count_total_aa - count_sheet
                                        fn_s3 = count_total_aa - count_coil

                                        '''
                                        print (count_total_aa)
                                        print (count_helix)
                                        print(count_sheet)
                                        print(count_coil)
                                        print(this_count_helix)
                                        print(this_count_sheet)
                                        print(this_count_coil)
                                        print("------")'''

                                        Ih1 = math.log(fsr1/fn_sr1) + math.log(fn_s1/fs1) + Ih1
                                        Is1 = math.log(fsr2/fn_sr2) + math.log(fn_s2/fs2) + Is1
                                        Ic1 = math.log(fsr3/fn_sr3) + math.log(fn_s3/fs3) + Ic1

                                        # Printing self information
                                        #print(Ih)
                                        #print(Is)
                                        #print(Ic)
                                        #print("--------")

                    #calculation pair information if left side we have less than 8 resides and right side also less than 8
                    elif count < 8:
                        for i in range(1,calculation + 1):
                            if i!= count:
                                if arrayforprotein[i][3] != 'null':
                                        aminoacid = arrayforprotein[i][3]
                                        count_total_aa = 0
                                        count_helix = 0
                                        count_sheet = 0
                                        count_coil = 0
                                        this_count_helix = 0
                                        this_count_sheet = 0
                                        this_count_coil = 0
                                        for line in datafromfile1:
                                            if line[0]!= protein_id:
                                                count_total_aa = count_total_aa + 1
                                                if line[4]=='Helix':
                                                    count_helix = count_helix + 1
                                                if line[4]=='Beta':
                                                    count_sheet = count_sheet + 1
                                                if line[4]=='Coil':
                                                    count_coil = count_coil + 1

                                                if line[4] == 'Helix' and line[3] == aminoacid:
                                                    this_count_helix = this_count_helix + 1
                                                elif line[4] == 'Beta' and line[3] == aminoacid:
                                                    this_count_sheet = this_count_sheet + 1
                                                elif line[4] == 'Coil' and line[3] == aminoacid:
                                                    this_count_coil = this_count_coil + 1

                                        fsr1 = this_count_helix
                                        fsr2 = this_count_sheet
                                        fsr3 = this_count_coil

                                        fsr_total = fsr1 + fsr2 + fsr3

                                        fn_sr1 = fsr_total - this_count_helix
                                        fn_sr2 = fsr_total - this_count_sheet
                                        fn_sr3 = fsr_total - this_count_coil

                                        fs1 = count_helix
                                        fs2 = count_sheet
                                        fs3 = count_coil

                                        fn_s1 = count_total_aa - count_helix
                                        fn_s2 = count_total_aa - count_sheet
                                        fn_s3 = count_total_aa - count_coil

                                        '''
                                        print (count_total_aa)
                                        print (count_helix)
                                        print(count_sheet)
                                        print(count_coil)
                                        print(this_count_helix)
                                        print(this_count_sheet)
                                        print(this_count_coil)
                                        print("------")'''

                                        Ih1 = math.log(fsr1/fn_sr1) + math.log(fn_s1/fs1) + Ih1
                                        Is1 = math.log(fsr2/fn_sr2) + math.log(fn_s2/fs2) + Is1
                                        Ic1 = math.log(fsr3/fn_sr3) + math.log(fn_s3/fs3) + Ic1

                                        # Printing self information
                                        #print(Ih)
                                        #print(Is)
                                        #print(Ic)
                                        #print("--------")


                    print ("Pair Information %f, %f, %f  " % (Ih1 , Is1, Ic1))
                    IH = Ih + Ih1
                    IS = Is + Is1
                    IC = Ic + Ic1


                    print ("Total I %f, %f, %f  " % (IH , IS, IC))

                    if (IH > IS) and (IH > IC):
                           largest = "H"
                    elif (IS > IH) and (IS > IC):
                           largest = "E"
                    else:
                           largest = "C"
                    print (largest)
                    print("-----")

                    final_dssp.append(largest)
            
                    
            
            
        

24862
15618
29983
Self Information 0.032993, 0.803405, -0.793804  
1
Pair Information 1.849941, -1.326574, -1.357204  
Total I 1.882933, -0.523169, -2.151007  
H
-----
24862
15618
29983
Self Information -0.383487, 0.295489, 0.108975  
2
Pair Information 1.249417, -0.011763, -1.943278  
Total I 0.865930, 0.283725, -1.834303  
H
-----
24862
15618
29983
Self Information 0.250030, -0.215918, -0.098754  
3
Pair Information 0.616762, -1.625212, -0.229156  
Total I 0.866792, -1.841129, -0.327909  
H
-----
24862
15618
29983
Self Information -0.165518, 0.930343, -0.721504  
4
Pair Information -1.123672, -0.737155, 0.402929  
Total I -1.289190, 0.193188, -0.318576  
E
-----
24862
15618
29983
Self Information 0.526713, -0.424901, -0.262216  
5
Pair Information -1.534568, -0.498212, 0.575059  
Total I -1.007855, -0.923114, 0.312843  
C
-----
24862
15618
29983
Self Information 0.554122, -0.368355, -0.325371  
6
Pair Information -1.585196, 0.115172, 0.116495  
Total I -1.031073, -0.253183, -0.208876

KeyboardInterrupt: 